In [5]:
import data_preparator as dp
import pandas

csv = pandas.read_csv('maestro-v3.0.0/maestro-v3.0.0.csv', sep=',', header=0).to_numpy()
filenames = [('maestro-v3.0.0/' + fn, composer) for composer, _, _, _, fn, _, _ in csv ]
composers = pandas.read_csv('new_composers.csv', sep=';').to_numpy()
composers_dict = {name: style for name, style in composers}

loaded_files_counter = 0
x = []
y = []
for filename, composer in filenames:
    loaded_files_counter += 1
    if loaded_files_counter % 10 == 0:
        print('Files loaded: ', loaded_files_counter, '/', len(filenames))
    score_parts_to_add = dp.get_score_parts(filename.replace('.midi', '.musicxml'), is_single=True)
    x_part = dp.get_input_data(score_parts_to_add)
    x += x_part.tolist()
    for i in range(0, len(x_part)):
        y.append(composers_dict[composer])


Files loaded:  10 / 1276
Files loaded:  20 / 1276
Files loaded:  30 / 1276
Files loaded:  40 / 1276
Files loaded:  50 / 1276
Files loaded:  60 / 1276
Files loaded:  70 / 1276
Files loaded:  80 / 1276
Files loaded:  90 / 1276
Files loaded:  100 / 1276
Files loaded:  110 / 1276
Files loaded:  120 / 1276
Files loaded:  130 / 1276
Files loaded:  140 / 1276
Files loaded:  150 / 1276
Files loaded:  160 / 1276
Files loaded:  170 / 1276
Files loaded:  180 / 1276
Files loaded:  190 / 1276
Files loaded:  200 / 1276
Files loaded:  210 / 1276
Files loaded:  220 / 1276
Files loaded:  230 / 1276
Files loaded:  240 / 1276
Files loaded:  250 / 1276
Files loaded:  260 / 1276
Files loaded:  270 / 1276
Files loaded:  280 / 1276
Files loaded:  290 / 1276
Files loaded:  300 / 1276
Files loaded:  310 / 1276
Files loaded:  320 / 1276
Files loaded:  330 / 1276
Files loaded:  340 / 1276
Files loaded:  350 / 1276
Files loaded:  360 / 1276
Files loaded:  370 / 1276
Files loaded:  380 / 1276
Files loaded:  390 / 

In [6]:
import tensorflow as tf
from tensorflow.keras import layers, losses
from tensorflow.keras.models import Model
import constants as c

latent_dim = 32

x_train = x[:round(len(x)*0.1)]
x_test = x[round(len(x)*0.1):round(len(x)*0.2)]
x_vad = x[round(len(x)*0.2):round(len(x)*0.3)]
y_train = y[:round(len(y)*0.1)]
y_test = y[round(len(y)*0.1):round(len(y)*0.2)]
y_vad = x[round(len(x)*0.2):round(len(x)*0.3)]

In [7]:
from keras.applications import resnet_v2

model = resnet_v2.ResNet50V2(weights=None, input_shape=(100, 12, 1))
model.compile(optimizer='adam', loss=losses.binary_crossentropy)
model.fit(x_train, x_train,
                epochs=20,
                shuffle=True,
                batch_size=10,
                validation_data=(x_test, x_test))

ValueError: Input size must be at least 32x32; got `input_shape=(100, 12, 1)`

In [9]:
class Autoencoder(Model):
  def __init__(self, latent_dim):
    super(Autoencoder, self).__init__()
    self.latent_dim = latent_dim
    self.encoder = tf.keras.Sequential([
      layers.Flatten(),
      layers.Dense(c.INPUT_MAX_WIDTH*12, activation='relu'),
      layers.ReLU(),
      layers.Dense(c.INPUT_MAX_WIDTH*3, activation='linear'),
    ])
    self.decoder = tf.keras.Sequential([
      layers.Dense(c.INPUT_MAX_WIDTH * c.INPUT_MAX_DEPTH, activation='sigmoid'),
      layers.ReLU(),
      layers.Reshape((c.INPUT_MAX_WIDTH, c.INPUT_MAX_DEPTH))
    ])

  def call(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded


autoencoder = Autoencoder(latent_dim)


autoencoder.compile(optimizer='adam', loss=losses.binary_crossentropy)
autoencoder.fit(x_train, x_train,
                epochs=20,
                shuffle=True,
                batch_size=10,
                validation_data=(x_test, x_test))


Epoch 1/20
593/593 [==============================] - 61s 102ms/step - loss: 0.2073 - val_loss: 0.1497

In [ ]:
from sklearn.neural_network import MLPClassifier

enc_some_x = autoencoder.encoder(x_vad).numpy()
X_mlp = enc_some_x
Y_mlp = y_vad


mlp = MLPClassifier(random_state=1, max_iter=100, activation='relu', hidden_layer_sizes=50)
ewrt = round(len(X_mlp)*0.9)
mlp.fit(X_mlp[:ewrt], Y_mlp[:ewrt])
print(mlp.score(X_mlp[ewrt:], Y_mlp[ewrt:]))
k=0
for x in X_mlp[ewrt:]:
  print(mlp.predict([x]), ' mialo byc ', Y_mlp[ewrt+k])
  k += 1
